# Database and Information System Characterising Objects in Space

[ESA DISCOS Website](https://discosweb.esoc.esa.int/)

## Code

### Setting up python

In [3]:
#calling pandas for data manipulation
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as snsclass_counts
#calling pprint and Request for API Calls
from pprint import pprint
import requests

API Setup info https://discosweb.esoc.esa.int/apidocs/v2

### Setting up API Connection

Test

In [4]:
#Connection to data souce API
URL = 'https://discosweb.esoc.esa.int'
token = 'ImUwYzQ2YTk1LWIzODQtNDVlYi1hNjUzLTAzZjAxYzEwYjI5NSI.A2EkjCeiipEzbpiP1hybmPOAP3o'

# Calling API and retriving data
response = requests.get(
    f'{URL}/api/objects',
    headers={
        'Authorization': f'Bearer {token}',
        'DiscosWeb-Api-Version': '2',
    },
    params={
        #Filtering data
        'filter': "eq(objectClass,Payload)&gt(reentry.epoch,epoch:'2020-01-01')",
        'sort': '-reentry.epoch',
    },
)

doc = response.json()

#checking data and outputing it
if response.ok:
    pprint('everything okay')
    print(doc)
else:

    pprint(doc['errors'])

'everything okay'
{'data': [{'type': 'object', 'attributes': {'xSectMax': 23.657, 'vimpelId': None, 'xSectAvg': 13.5615, 'name': 'Starlink 1118', 'diameter': None, 'depth': 1.5, 'mass': 260.0, 'span': 8.86, 'cosparId': '2020-001BB', 'satno': 44963, 'height': 0.1, 'width': 3.7, 'xSectMin': 0.2311, 'shape': 'Box + 1 Pan', 'objectClass': 'Payload'}, 'relationships': {'reentry': {'links': {'self': '/api/objects/62184/relationships/reentry', 'related': '/api/objects/62184/reentry'}}, 'operators': {'links': {'self': '/api/objects/62184/relationships/operators', 'related': '/api/objects/62184/operators'}}, 'launch': {'links': {'self': '/api/objects/62184/relationships/launch', 'related': '/api/objects/62184/launch'}}, 'destinationOrbits': {'links': {'self': '/api/objects/62184/relationships/destination-orbits', 'related': '/api/objects/62184/destination-orbits'}}, 'initialOrbits': {'links': {'self': '/api/objects/62184/relationships/initial-orbits', 'related': '/api/objects/62184/initial-orbi

Full pull of data **TAKES A BIT TO RUN**

In [5]:
#"https://discosweb.esoc.esa.int/api/objects?filter=eq(objectClass,Payload)"

#response = response.get('https://discosweb.esoc.esa.int/api/objects?filter=eq(objectClass,Payload)')

page = 1
doc = {}

while(page<263):# 262 total pages
  response = requests.get(
      f'{URL}/api/objects?page%5Bsize%5D=100&page%5Bnumber%5D={page}',
      headers={
          'Authorization': f'Bearer {token}',
          'DiscosWeb-Api-Version': '2',
      },
      params={
          'filter': "and(cosparId=ne=null,mass=ne=null)"
      }
  )
  if response.status_code == 200:
      data = response.json()
      doc[page] = data
  #else:
      #print(f"Error: {response.status_code}")
      #break

  page += 1

#print(doc)



### Converting response doc to a Dataframe

In [6]:
data_list = [item['attributes'] for page_data in doc.values() for item in page_data['data']]

df = pd.DataFrame(data_list)

df


df.to_csv('output.csv', index=False)

#SQL

In [2]:
import mysql.connector
connection = mysql.connector.connect(
    host='localhost',
    user='babb',
    password='C^3M8fsvXbaW',
    database='DISCOS'
)

# Create cursor
cursor = connection.cursor()

# Execute SQL query
sql = "SELECT * FROM Objects"
cursor.execute(sql)

# Fetch results from SQL query
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

# Close the connection
connection.close()

DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (10061)

In [ ]:
import mysql.connector
import pandas as pd

# Establish connection
connection = mysql.connector.connect(
    host='localhost',
    user='your_username',
    password='your_password',
    database='your_database'
)


cursor = connection.cursor()

sql = "SELECT * FROM Objects"
cursor.execute(sql)

results = cursor.fetchall()

columns = [column[0] for column in cursor.description]
sql_df = pd.DataFrame(results, columns=columns)

differences = df.merge(sql_df, indicator=True, how='left')
differences = differences[differences['_merge'] == 'left_only']

if not differences.empty:
    insert_cursor = connection.cursor()

    for _, row in differences.iterrows():
        values = tuple(row.values)

        insert_sql = "INSERT INTO Objects (column1, column2, column3) VALUES (%s, %s, %s)"

        insert_cursor.execute(insert_sql, values)

    connection.commit()
    insert_cursor.close()

connection.close()